#### Reasoning without Observation

ReWOO proposes an agent that combines a multi-step planner and variable substitution for effective tool use. It was designed to improve the ReACT-style agent architecture in the following ways:
- Reduce token consumption and execution time by generating the full chain of tools used in a single pass.
- Simplify the fine-tuning process. Since the planning doesn't depend on the outputs of the tool, models can be fine-tuned without actually involving the tools(in theory).

ReWOO is made of 3 modules:
1. Planner: GEnrate the plan in the format of: plan / reason, tool
2. Worker: execute the tool with the provided arguments
3. Solver: generates the answer for the initial task based on the tool observations.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing import TypedDict, List


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

##### Planner
The planner prompts an LLM to generate a plan in the form of a task list. The arguments to each task are strings that may contain special variables that are used for variable substitution from other task results.


In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = "mistralai/Mixtral-8x7B-Instruct-v0.1")

In [4]:
prompt = """
For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
less than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}
"""

In [7]:
task = "What is the hometown of the 2024 australian open winner"
result = model.invoke(prompt.format(task=task))

In [8]:
print(result.content)

 Plan: Use Google to find out who the winner of the 2024 Australian Open is. #E1 = Google[2024 Australian Open winner]
Plan: Utilize LLM to extract the name of the winner from the Google search results. #E2 = LLM[Who won the 2024 Australian Open, given #E1?]
Plan: Perform another Google search to find the hometown of the identified tennis player. #E3 = Google[Hometown of #E2]
Plan: Lastly, use LLM to determine the hometown of the 2024 Australian Open winner from the Google search results. #E4 = LLM[What is the hometown of #E2, given #E3?]


##### Planner Node
To connect the planner to our graph, we will creat a `get_plan` node that accepts the `ReWOO` state and returns with a state update for the `steps` and `plan_string` fields.

In [17]:
import re
from langchain_core.prompts import ChatPromptTemplate

regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model

def get_plan(state: ReWOO):
    task = state["task"]
    result = planner.invoke({"task": task})
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}

##### Executor 
The executor receives the plan and executes the tools in sequence. 
Below, initiate the search engine and define the tool execution node.

In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=1)

In [19]:
def _get_current_task(state: ReWOO):
    if state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1
    
    
def tool_execution(state: ReWOO):
    """
    Worker node that executes the tools of a given plan
    """
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = state["results"] or {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
        
    if tool == "Google":
        result = search.invoke(tool_input)
    elif tool == "LLM":
        result = model.invoke(tool_input)
    
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {
        "results": _results
    }

##### Solver
The solver receives the full plan and generates the final response based on the responses of the tool calls from the worker

In [20]:
solver_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""

def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = state["results"] or {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
        plan += f"Plan: {_plan} \n {step_name} = {tool}[{tool_input}]"
    prompt = solver_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)
    return {"result": result.content}

##### Define Graph
Our graph defines the workflow. Each of the planner, tool executor, and solver modules are added as nodes.

In [21]:
def _route(state: ReWOO):
    _step = _get_current_task(state)
    if _step is None:
        return "solve"
    else:
        return "tool"

In [22]:
from langgraph.graph import StateGraph, END

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)

graph.add_edge("plan", "tool")
graph.add_edge("solve", END)

graph.add_conditional_edges("tool", _route)
graph.set_entry_point("plan")

app = graph.compile()

In [23]:
for s in app.stream({"task" : task}):
    print(s)
    print("---")

{'plan': {'steps': [('Use Google to find out who the 2024 Australian Open winner is, since the winner is not known yet. ', '#E1', 'Google', '2024 Australian Open winner'), ('Once the winner is identified, use Google again to find out the hometown of the winner. ', '#E2', 'Google', 'Hometown of #E1')], 'plan_string': ' Plan: Use Google to find out who the 2024 Australian Open winner is, since the winner is not known yet. #E1 = Google[2024 Australian Open winner]\nPlan: Once the winner is identified, use Google again to find out the hometown of the winner. #E2 = Google[Hometown of #E1]\n\nFollowing these steps, you will first determine the winner of the 2024 Australian Open and then find out their hometown using Google searches.'}}
---
{'tool': {'results': {'#E1': "[{'url': 'https://www.reuters.com/sports/tennis/sinner-savours-rotterdam-success-after-australian-open-high-2024-02-19/', 'content': 'Sinner, who lifted his first major at Melbourne Park by defeating Daniil Medvedev in the fin